## 0. Setup

In [1]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import IterativeStratification
from imblearn.under_sampling import RandomUnderSampler
from copy import deepcopy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pd.options.display.max_columns = 999

In [5]:
# # current directory
# current_dir = os.getcwd()

# # two folders up
# parent_dir = os.path.dirname(os.path.dirname(current_dir))

# # data directory
# data_dir_processed = os.path.join(parent_dir, "data/processed")

# data directory (GDrive)
data_dir_processed = '/content/drive/MyDrive/DATASCI_210/data/processed/'

In [6]:
def return_column_values_sum_and_percentage(dataframe_input, column_input):
    total_sum = dataframe_input[column_input].sum()
    percentages = dataframe_input[column_input] / total_sum
    sums_percentages = pd.DataFrame({
        'sum': dataframe_input[column_input],
        'percentage': percentages
    })
    sums_percentages['cumsum_percentage'] = sums_percentages['percentage'].cumsum()
    # sums_percentages['sum'] = sums_percentages['sum'].apply(lambda x: "{:,}".format(x))
    sums_percentages['percentage'] = sums_percentages['percentage'].mul(100).round(1).astype(str) + '%'
    sums_percentages['cumsum_percentage'] = sums_percentages['cumsum_percentage'].mul(100).round(1).astype(str) + '%'
    return sums_percentages

## 1. Data Acquisition

In [11]:
# Define the names of the JSON files
chexpert_final_df_filename = 'chexpert_all_findings.json'

# Load the JSON files into a DataFrame
with open(data_dir_processed + chexpert_final_df_filename) as chexpert_json_file:
    chexpert_dict_file = json.load(chexpert_json_file)

# Converting json dataset from dictionary to dataframe
chexpert_final_df = pd.DataFrame.from_dict(chexpert_dict_file)
chexpert_final_df.reset_index(drop=True, inplace=True)

## 2. Data Preprocessing

In [12]:
chexpert_final_df.head()

,patient_id,visit_id,study_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,atelectasis,cardiomegaly,consolidation,edema,enlarged_cardiomediastinum,fracture,lung_lesion,lung_opacity,no_finding,pleural_effusion,pleural_other,pneumonia,pneumothorax,support_devices,positive_label_total,finding_names,radiology_note,discharge_note,chief_complaint,major_surgical_or_invasive_procedure,history_of_present_illness,past_medical_history,family_history
0,10000032.0,22595853,50414267.0,98.4,70.0,16.0,97.0,106.0,63.0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,no_finding,FINAL REPORT\...,\nName: ___ Unit No: _...,Worsening ABD distension and pain,Paracentesis,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis \n2. No history of abnormal ...,"She a total of five siblings, but she is not ..."
1,10000935.0,21738619,56164612.0,97.8,120.0,20.0,98.0,121.0,77.0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"lung_lesion, lung_opacity",FINAL REPORT\...,\nName: ___ Unit No: ___...,"nausea, vomiting, cough",none,"___ female, with past medical history signific...",PMH: \n# high grade SBO ___ s/p exploratory la...,non contributory
2,10000980.0,29654838,59988438.0,97.8,57.0,18.0,100.0,180.0,88.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,pleural_effusion,FINAL REPORT\...,\nName: ___ Unit No: ___\n \nAdmi...,Shortness of breath,None,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",Denies cardiac family history. Family hx of DM...
3,10001176.0,23334588,53186264.0,101.3,97.0,18.0,93.0,168.0,58.0,6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,lung_opacity,FINAL REPORT\...,\nName: ___ Unit No: ___...,fever,none,"___ with history of morbid obesity, coronary a...",MYOCARDIAL INFARCT - INFEROPOSTERIOR \nHYPERC...,Non contributory
4,10001217.0,24597018,52067803.0,99.0,81.0,16.0,97.0,160.0,102.0,0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,atelectasis,FINAL REPORT\...,\nName: ___ Unit No: ___\n \n...,"Left hand and face numbness, left hand weaknes...",Right parietal craniotomy for abscess incision...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,"Mother with pancreatic cancer, brother-lung ca..."


In [13]:
return_column_values_sum_and_percentage(chexpert_final_df.groupby("finding_names").agg({"study_id": "count"}).sort_values("study_id", ascending=False), "study_id")

,sum,percentage,cumsum_percentage
finding_names,,,
no_finding,7863,48.4%,48.4%
lung_opacity,1178,7.3%,55.7%
cardiomegaly,689,4.2%,59.9%
atelectasis,644,4.0%,63.9%
pleural_effusion,418,2.6%,66.5%
...,...,...,...
"atelectasis, cardiomegaly, enlarged_cardiomediastinum, pleural_effusion, pneumonia",1,0.0%,100.0%
"cardiomegaly, fracture, support_devices",1,0.0%,100.0%
"cardiomegaly, lung_lesion, lung_opacity, pleural_effusion",1,0.0%,100.0%


In [14]:
findings_columns = [
 'atelectasis',
 'cardiomegaly',
 'consolidation',
 'edema',
 'enlarged_cardiomediastinum',
 'fracture',
 'lung_lesion',
 'lung_opacity',
 'no_finding',
 'pleural_effusion',
 'pleural_other',
 'pneumonia',
 'pneumothorax',
 'support_devices',
]

pathology_dict = {}

for col in findings_columns:
  value_zero = chexpert_final_df[chexpert_final_df[col] == 0][col].count()
  value_positive = chexpert_final_df[chexpert_final_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["pathology", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)

,pathology,not_mention,positive_mention,total_studies
0,no_finding,8059,8175,16234
1,lung_opacity,12933,3301,16234
2,atelectasis,14089,2145,16234
3,pleural_effusion,14188,2046,16234
4,cardiomegaly,14581,1653,16234
5,edema,14973,1261,16234
6,pneumonia,15079,1155,16234
7,support_devices,15389,845,16234
8,lung_lesion,15637,597,16234
9,consolidation,15843,391,16234


In [15]:
excluded_target_variables = [
#  'atelectasis',
#  'cardiomegaly',
 'consolidation',
 'edema',
 'enlarged_cardiomediastinum',
 'fracture',
 'lung_lesion',
#  'lung_opacity',
#  'no_finding',
#  'pleural_effusion',
 'pleural_other',
 'pneumonia',
 'pneumothorax',
 'support_devices',
]

# Mask to include the selected findings (top 4 + "no_finding")
mask = chexpert_final_df['finding_names'].str.contains('|'.join(excluded_target_variables), na=False)

# Filter dataset to only include the selected findings (top 4 + "no_finding")
chexpert_filtered_final_df = chexpert_final_df[~mask]

In [17]:
# Target variables
target_variables = [
    'atelectasis',
    'cardiomegaly',
    'lung_opacity',
    'pleural_effusion',
]

In [18]:
pathology_dict = {}

for col in target_variables:
  value_zero = chexpert_filtered_final_df[chexpert_filtered_final_df[col] == 0][col].count()
  value_positive = chexpert_filtered_final_df[chexpert_filtered_final_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pathology_dict["no_finding"] = ["no_finding", 0, chexpert_filtered_final_df[chexpert_filtered_final_df["finding_names"] == 'no_finding'].shape[0], chexpert_filtered_final_df.shape[0]]

pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["finding", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)

,finding,not_mention,positive_mention,total_studies
0,no_finding,0,7863,11997
1,lung_opacity,10060,1937,11997
2,atelectasis,10506,1491,11997
3,pleural_effusion,10883,1114,11997
4,cardiomegaly,10985,1012,11997


In [19]:
# Eliminate "no_finding" column
chexpert_filtered_final_df = chexpert_filtered_final_df.drop("no_finding", axis=1)

# Eliminate columns that are not needed
chexpert_filtered_final_df = chexpert_filtered_final_df.drop(excluded_target_variables, axis=1)

In [20]:
chexpert_filtered_final_df[chexpert_filtered_final_df.finding_names != 'no_finding'].positive_label_total.value_counts()

1.0    2929
2.0    1001
3.0     193
4.0      11
Name: positive_label_total, dtype: int64

In [21]:
# Only consider those observations that have less than 3 labels
chexpert_filtered_final_df = chexpert_filtered_final_df[chexpert_filtered_final_df.positive_label_total <= 2]

In [22]:
pathology_dict = {}

for col in target_variables:
  value_zero = chexpert_filtered_final_df[chexpert_filtered_final_df[col] == 0][col].count()
  value_positive = chexpert_filtered_final_df[chexpert_filtered_final_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pathology_dict["no_finding"] = ["no_finding", 0, chexpert_filtered_final_df[chexpert_filtered_final_df["finding_names"] == 'no_finding'].shape[0], chexpert_filtered_final_df.shape[0]]

pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["finding", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)

,finding,not_mention,positive_mention,total_studies
0,no_finding,0,7863,11793
1,lung_opacity,10027,1766,11793
2,atelectasis,10488,1305,11793
3,pleural_effusion,10861,932,11793
4,cardiomegaly,10865,928,11793


In [23]:
return_column_values_sum_and_percentage(chexpert_filtered_final_df.groupby("finding_names").agg({"study_id": "count"}).sort_values("study_id", ascending=False), "study_id")

,sum,percentage,cumsum_percentage
finding_names,,,
no_finding,7863,66.7%,66.7%
lung_opacity,1178,10.0%,76.7%
cardiomegaly,689,5.8%,82.5%
atelectasis,644,5.5%,88.0%
pleural_effusion,418,3.5%,91.5%
"atelectasis, lung_opacity",326,2.8%,94.3%
"atelectasis, pleural_effusion",268,2.3%,96.5%
"lung_opacity, pleural_effusion",168,1.4%,98.0%
"cardiomegaly, lung_opacity",94,0.8%,98.8%


In [24]:
chexpert_filtered_final_df.to_json(data_dir_processed + '/' + 'chexpert_selected_4_findings.json')

In [ ]:
# Separate features and target variables
X = chexpert_filtered_final_df.drop(target_variables, axis=1)  # Features
y = chexpert_filtered_final_df[target_variables]  # Target variables

# First, split the data into a training set and a temporary set (combining test and validation)
stratifier = IterativeStratification(n_splits=3, order=1)
train_indexes, temp_indexes = next(stratifier.split(X.values, y.values))

# First, split the data into a training set and a temporary set (combining test and validation)
X_train, y_train = X.iloc[train_indexes], y.iloc[train_indexes]
X_temp, y_temp = X.iloc[temp_indexes], y.iloc[temp_indexes]

# Split the temporary set into actual test and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp.values)

# Recombine features and target for each set
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
val_df = pd.concat([X_val.reset_index(drop=True), y_val.reset_index(drop=True)], axis=1)
test_df = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

In [ ]:
pathology_dict = {}

for col in target_variables:
  value_zero = train_df[train_df[col] == 0][col].count()
  value_positive = train_df[train_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pathology_dict["no_finding"] = ["no_finding", 0, train_df[train_df["finding_names"] == 'no_finding'].shape[0], train_df.shape[0]]

train_distribution = pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["finding", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)
train_distribution["percentage"] = round(train_distribution["positive_mention"] / train_distribution["total_studies"], 2)*100
train_distribution

,finding,not_mention,positive_mention,total_studies,percentage
0,no_finding,0,5233,8823,59.0
1,lung_opacity,7428,1395,8823,16.0
2,atelectasis,7909,914,8823,10.0
3,pleural_effusion,8069,754,8823,9.0
4,cardiomegaly,8086,737,8823,8.0
5,edema,8328,495,8823,6.0
6,pneumonia,8343,480,8823,5.0


In [ ]:
pathology_dict = {}

for col in target_variables:
  value_zero = val_df[val_df[col] == 0][col].count()
  value_positive = val_df[val_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pathology_dict["no_finding"] = ["no_finding", 0, val_df[val_df["finding_names"] == 'no_finding'].shape[0], val_df.shape[0]]

val_distribution = pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["finding", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)
val_distribution["percentage"] = round(val_distribution["positive_mention"] / val_distribution["total_studies"], 2)*100
val_distribution

,finding,not_mention,positive_mention,total_studies,percentage
0,no_finding,0,1315,2206,60.0
1,lung_opacity,1857,349,2206,16.0
2,atelectasis,1977,229,2206,10.0
3,pleural_effusion,2018,188,2206,9.0
4,cardiomegaly,2021,185,2206,8.0
5,edema,2084,122,2206,6.0
6,pneumonia,2087,119,2206,5.0


In [ ]:
pathology_dict = {}

for col in target_variables:
  value_zero = test_df[test_df[col] == 0][col].count()
  value_positive = test_df[test_df[col] == 1][col].count()
  total_studies = value_zero + value_positive
  pathology_dict[col] = [col, value_zero, value_positive, total_studies]

pathology_dict["no_finding"] = ["no_finding", 0, test_df[test_df["finding_names"] == 'no_finding'].shape[0], test_df.shape[0]]

test_distribution = pd.DataFrame.from_dict(pathology_dict, orient="index", columns=["finding", "not_mention", "positive_mention", "total_studies"]).sort_values("positive_mention", ascending=False).reset_index(drop=True)
test_distribution["percentage"] = round(test_distribution["positive_mention"] / test_distribution["total_studies"], 2)*100
test_distribution

,finding,not_mention,positive_mention,total_studies,percentage
0,no_finding,0,1315,2206,60.0
1,lung_opacity,1858,348,2206,16.0
2,atelectasis,1978,228,2206,10.0
3,pleural_effusion,2017,189,2206,9.0
4,cardiomegaly,2022,184,2206,8.0
5,edema,2081,125,2206,6.0
6,pneumonia,2085,121,2206,5.0


In [ ]:
# Export dataframes to json files
train_df.to_json(data_dir_processed + 'train_set__chexpert__6_findings.json')
val_df.to_json(data_dir_processed + 'validation_set__chexpert__6_findings.json')
test_df.to_json(data_dir_processed + 'test_set__chexpert__6_findings.json')